# Error Mitigation: Zero Noise Extrapolation (ZNE) for indirect-control system
---

_This notebook contains experimental results for Zero Noise Extrapolation (ZNE) in an indirect-control system._

Zero-Noise Extrapolation (ZNE) is one of the most widely used techniques in quantum error mitigation to estimate the expectation value of an observable under noise-free conditions. In ZNE, noise is intentionally introduced into the quantum circuit, and the expectation value of the target observable is measured at different noise levels. Once sufficient data is collected, we extrapolate this data to the zero-noise limit.

There are several strategies for zero-noise extrapolation, with the Richardson extrapolation technique being perhaps the most common. Here, we adapted the multivariate framework of Richardson extrapolation as discussed in the paper [_"Quantum error mitigation by layerwise Richardson extrapolation" by Vincent Russo and Andrea Mari (arXiv:2402.04000, 2024)_](https://arxiv.org/abs/2402.04000).

### A. The code for multivariate Richardson extrapolation

In [23]:
"""
We have used the multivariate framework for Richardson extrapolation as discussed in the paper "Quantum error mitigation by layerwise Richardson extrapolation" by Vincent Russo and Andrea Mari (arXiv:2402.04000, 2024).

Parts of the following code are adapted from their notebook, which can be found at the following GitHub repository: https://github.com/unitaryfund/research/blob/main/lre/layerwise_richardson_extrapolation.ipynb.
"""

import numpy as np
from collections import Counter
import itertools
 

class ZeroNoiseExtrapolation:
    def __init__(self, dataPoints: list[tuple[float]], degree: int):
        """
        Initialize with a list of data points, each represented as a tuple of floats.
        """
        self.dataPoints = dataPoints
        self.degree = degree

        self.NoiseData = [tuple(point[:3]) for point in self.dataPoints]
        self.ExpectationVals = [point[-1] for point in dataPoints]

    def getRichardsonZNE(self):

        RichardsonZNEval = 0

        sampleMatrix = self.sample_matrix(sample_points = self.NoiseData, degree = self.degree) # type: ignore
        detA = np.linalg.det(sampleMatrix)

        matrices = self.generate_modified_matrices(sampleMatrix) # type: ignore

        for E, matrix in zip(self.ExpectationVals, matrices):
            RichardsonZNEval += E * (np.linalg.det(matrix)/detA)
        
        return RichardsonZNEval
     
    @staticmethod
    def get_monomials(n: int, d: int) -> list[str]:
        """
        Compute monomials of degree `d` in graded lexicographical order.
        """
        variables = [f"λ_{i}" for i in range(1, n + 1)]
        
        monomials = []
        for degree in range(d, -1, -1):
            # Generate combinations for the current degree
            combos = list(itertools.combinations_with_replacement(variables, degree))
            
            # Sort combinations lexicographically
            combos.sort()
            
            # Construct monomials from sorted combinations
            for combo in combos:
                monomial_parts = []
                counts = Counter(combo)
                # Ensure variables are processed in lexicographical order
                for var in sorted(counts.keys()):
                    count = counts[var]
                    if count > 1:
                        monomial_parts.append(f"{var}**{count}")
                    else:
                        monomial_parts.append(var)
                monomial = "*".join(monomial_parts)
                # Handle the case where degree is 0
                if not monomial:
                    monomial = "1"
                monomials.append(monomial)
        # "1" should be the first monomial. Note that order d > c > b > a means vector of monomials = [a, b, c, d].            
        return monomials[::-1]

    @staticmethod
    def sample_matrix(sample_points: list[int], degree: int) -> np.ndarray:
        """Construct a matrix from monomials evaluated at sample points."""
        n = len(sample_points[0])  # type: ignore # Number of variables based on the first sample point
        monomials = ZeroNoiseExtrapolation.get_monomials(n, degree) # type: ignore
        matrix = np.zeros((len(sample_points), len(monomials)))

        for i, point in enumerate(sample_points):
            for j, monomial in enumerate(monomials):
                var_mapping = {f"λ_{k+1}": point[k] for k in range(n)} # type: ignore
                matrix[i, j] = eval(monomial, {}, var_mapping)
        return matrix

    @staticmethod
    def generate_modified_matrices(matrix):
        """
        It generates the Mi(0) matreces for i = 1 to length of sample matrix.
        See this papaer for the detail mathematical formalism: 
        "Quantum error mitigation by layerwise Richardson extrapolation" by Vincent Russo, Andrea Mari, https://arxiv.org/abs/2402.04000
        """
        n = len(matrix)  # Size of the square matrix
        identity_row = [1] + [0] * (n - 1)  # Row to replace with

        modified_matrices = []
        determinants = []
        for i in range(n):
            # Create a copy of the original matrix
            modified_matrix = np.copy(matrix)
            # Replace the i-th row with the identity_row
            modified_matrix[i] = identity_row
            modified_matrices.append(modified_matrix)
            # Calculate the determinant of the modified matrix
            determinant = np.linalg.det(modified_matrix)
            determinants.append(determinant)
        
        return modified_matrices    
    

### B. Experimental result

In our noisy indirect circuit we can introduce different noise level by adjusting three noise factor. Therefore we have three independent variables and one dependent variable (i.e. the expectation value).


#### 5-qubit system

For a 5-qubit system, Indirect-VQE yields satisfactory results with a depth of 10 layers in a noise-free circuit. Consequently, for the noisy circuit, we have selected the same circuit depth.

The noise probabilities for the noisy circuit are ( in the config file `circuit.noise.value : [0.0001, 0.0001, 0.0001, 0.0001]`

First, we ran the Indirect-VQE with noisy circuit, but without any redundent identities (`circuit.noise.factor: [0, 0, 0`]) and obtain the optimized parameters (`optimization.status: True`).

In [27]:
# Data for 5-qubit, 10 layers system
data_5Q = [
    (4, 1, 0, -3.916268577812803),  # noise factor [ 0, 0, 0 ]
    (12, 1, 3, -3.891039542364015), # noise factor [ 1, 0, 0 ]
    (12, 3, 3, -3.8437860343347316), # noise factor [ 1, 1, 0 ]
    (12, 3, 9, -3.8100008383254043), # noise factor [ 1, 1, 1 ]
    (20, 1, 3, -3.86598644113986), # noise factor [ 2, 0, 0 ]
    (20, 3, 3, -3.819049193145926), # noise factor [ 2, 1, 0 ]
    (20, 3, 9, -3.7854901666397804), # noise factor [ 2, 1, 1 ]
    (20, 5, 3, -3.772707487765387), # noise factor [ 2, 2, 0 ]
    (20, 5, 9, -3.7067492774117636), # noise factor [ 2, 2, 1 ]
    (20, 5, 15, -3.642004799002842) # noise facfor [ 2, 2, 2 ]
]

zne_5Q = ZeroNoiseExtrapolation(dataPoints = data_5Q, degree = 2)
# Extrapolated value at (0, 0, 0)
zne_5Q_val = zne_5Q.getRichardsonZNE()
print(f"Richardson extrapolated value: {zne_5Q_val}")

Richardson extrapolated value: -3.943889509533736


The exact expectation value is -3.999999999999999. There for the ZNE is working properly.

#### 8-qubit system

Now 8-qubit system requires much deeper circuit compared to 5-qubit. 
Using the noise-less circuit, we have found that with 25 layers we get satisfactory result. Therefore we set the depeth of the noisy circuit to same.

The noise probabilities for the noisy circuit are ( in the config file `circuit.noise.value : [0.0001, 0.0001, 0.0001, 0.0001]`

First, we ran the Indirect-VQE with noisy circuit, but without any redundent identities (`circuit.noise.factor: [0, 0, 0`]) and obtain the optimized parameters (`optimization.status: True`).

In [ ]:
# Data for 8-qubit system
data_8Q = [
    
]

The following is the configuration for our indirect circuit.

```qubit = 8
layer = 25
noise probabilities = [0.0001, 0.0001, 0.0001, 0.0001]
exact solution = -7.000000000000004
```

We ran our indirect eigen solver at various noise level by changing the `factor` value inside the `config.yml`

In [22]:
data_sys1 = [
    (4, 1, 0, -6.742247461826676),
    (12, 3, 8, -6.405455124800368),
    (12, 5, 24, -5.839787462274414),
    (4, 7, 8, -5.991097523966149),
    (4, 7, 24, -5.533783365921837),
    (28, 9, 24, -5.005206166353052),
    (4, 9, 56, -4.206331147604706),
    (44, 1, 24, -6.357848309361182),
    (44, 3, 24, -5.9516342852267075),
    (4, 13, 8, -5.328222900577986)
]

zne_sys1 = ZeroNoiseExtrapolation(dataPoints= data_sys1, degree= 2)
zne_sys1.getRichardsonZNE()

-6.821098781914534

System #2

```qubit = 8
layer = 25
noise probabilities = [0.000001, 0.000001, 0.000001, 0.000001]
exact solution = -7.000000000000004
```


In [24]:
data_set2 = [
    (4, 1, 0, -6.254792723611204),
    (12, 3, 8, -6.250858745027363),
    (12, 5, 8, -6.2478883834002135),
    (4, 5, 24, -6.244810370105138),
    (20, 5, 24, -6.242889589082837),
    (4, 7, 24, -6.239825585489937),
    (12, 7, 40, -6.2328185853721925),
    (12, 1, 40, -6.253830617363461),
    (4, 11, 8, -6.23994604760344),
    (12, 11, 8, -6.238986354642458)
]

zne_set2 = ZeroNoiseExtrapolation(dataPoints = data_set2, degree = 2)
zne_set2.getRichardsonZNE()

-6.256252868096004